In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import Pipeline
import joblib

In [2]:
data = pd.read_csv('/content/Data4Model.csv')
data.sample(10)

,preprocessing,dialect
101713,تعرف واله بتكلم بجد انتوا محتاجين صلاح لاعب سر...,EG
4368,منوره غاده اله يعزك,LY
115308,الغالي انشاله بتشوف الا الهنا الف مبروك,LB
98974,وازاي دنيا مشوفتكيش النهارده معشتكيش انا كنت م...,EG
51057,دولا مين ودولا مين دولا عساكر مصرين دولا القوه...,EG
94672,ليه كدا راجنر مات حرقتيلي المسلسل كدا,EG
113268,شو عم بيصير بو صعب جبران باسيل مش قادره نام كو...,LB
95341,ده الموضوع طلع كبير,EG
82309,عارفه بجد يتبشر ويبقي جبنه موتزاريلا كده الناس...,EG
48849,كنت فين بدري عم المره الجيه بقي,EG


In [6]:
data

,preprocessing,dialect
0,قليلين ادب ومنافقين اختهم او قريبتهم تعاكس تقو...,LY
1,اليبين متقلبين بالنسبه ليا انا ميليشياوي زمان ...,LY
2,تانيه شاب ليبي بيرتاح لبنت مختلفه ويلاحظ انها ...,LY
3,رانيا عقليتك متخلفه اولا الانسان يلي يحتاج اهل...,LY
4,شكلك متعقده علشان الراجل تحبيه ازوج بنت يتيمه ...,LY
...,...,...
147720,الناس دي بتنفخ قربه مقدوده بالدارجي كده البلد ...,SD
147721,انت عايش وين بره السودان شنو ماشايف البحصل دا,SD
147722,ماحرم نفسي ميسي حريف ولعاب برضو مدريدي وافتخر,SD
147723,ياخي ديل ماخلو لشيطان وابليس شي يروحو وين ربنا...,SD


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147725 entries, 0 to 147724
Data columns (total 2 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   preprocessing  147633 non-null  object
 1   dialect        147725 non-null  object
dtypes: object(2)
memory usage: 2.3+ MB


In [5]:
print("Number of missing values in each column:")
print(data.isnull().sum())

# Drop rows with missing values
data.dropna(inplace=True)

# Verify that NaNs have been dropped
print("Number of missing values after dropping rows:")
print(data.isnull().sum())

Number of missing values in each column:
preprocessing    92
dialect           0
dtype: int64
Number of missing values after dropping rows:
preprocessing    0
dialect          0
dtype: int64


In [21]:
def custom_tokenizer(text):
    return text.split()

In [7]:
X = data['preprocessing']
y = data['dialect']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [22]:
modelML = Pipeline([
    ('tfidf', TfidfVectorizer(tokenizer=custom_tokenizer, lowercase=False)),
    ('clf', LogisticRegression(max_iter=1000, solver='lbfgs', multi_class='auto'))
])

In [23]:
modelML.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(lowercase=False,
                                 tokenizer=<function custom_tokenizer at 0x7c0445cde830>)),
                ('clf', LogisticRegression(max_iter=1000))])

In [24]:
y_pred = modelML.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Test Accuracy: {accuracy}')
print(classification_report(y_test, y_pred))


Test Accuracy: 0.8177261489484201
              precision    recall  f1-score   support

          EG       0.81      0.92      0.86     11487
          LB       0.87      0.81      0.84      5639
          LY       0.76      0.83      0.80      7240
          MA       0.92      0.61      0.73      2302
          SD       0.86      0.55      0.67      2859

    accuracy                           0.82     29527
   macro avg       0.84      0.74      0.78     29527
weighted avg       0.82      0.82      0.81     29527



In [25]:
new_data = [
    "الناس دي بتنفخ قربه مقدوده بالدارجي كده البلد دي ماعندهاش حل",  # Example from SD
    "رانيا عقليتك متخلفه اولا الانسان يلي يحتاج اهل ليشوف مصلحته ضعيف شخصية",  # Example from LY
]


In [26]:
new_data_transformed = [' '.join(map(str, sentence.split())) for sentence in new_data]

# Make predictions
predictions = modelML.predict(new_data_transformed)

# Print the predictions
for text, prediction in zip(new_data, predictions):
    print(f"Text: {text}\nPredicted Dialect: {prediction}\n")

Text: الناس دي بتنفخ قربه مقدوده بالدارجي كده البلد دي ماعندهاش حل
Predicted Dialect: EG

Text: رانيا عقليتك متخلفه اولا الانسان يلي يحتاج اهل ليشوف مصلحته ضعيف شخصية
Predicted Dialect: LB



In [27]:
import joblib

joblib.dump(modelML, 'sentiment_analysis_ml_model.joblib')

['sentiment_analysis_ml_model.joblib']

In [28]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Test Accuracy: {accuracy}')
print(classification_report(y_test, y_pred))

# Optionally, print some of the predictions alongside the true values for manual inspection
for i in range(10):  # Adjust the range to print more or fewer examples
    print(f"Text: {X_test.iloc[i]}")
    print(f"True Dialect: {y_test.iloc[i]}, Predicted Dialect: {y_pred[i]}")
    print()

Test Accuracy: 0.8177261489484201
              precision    recall  f1-score   support

          EG       0.81      0.92      0.86     11487
          LB       0.87      0.81      0.84      5639
          LY       0.76      0.83      0.80      7240
          MA       0.92      0.61      0.73      2302
          SD       0.86      0.55      0.67      2859

    accuracy                           0.82     29527
   macro avg       0.84      0.74      0.78     29527
weighted avg       0.82      0.82      0.81     29527

Text: خشيت عشان اتابع الشخصيه دي لقيتها متابعاني اصلا انو عامل الف ودقداق وده شي غريب عالم تويتر فض فوك يازول
True Dialect: SD, Predicted Dialect: SD

Text: الف سلامه عليكي وجع
True Dialect: EG, Predicted Dialect: EG

Text: جماعه الخير رسيل وملاك حرقولن حساباتهن متل كتار غيرهن ادعموهن ليلي بيحب ان شاله بيدعمونا وقت احراق حساباتنا
True Dialect: LB, Predicted Dialect: LB

Text: مبروك لرجاله دول الي يستحقو الزمالك
True Dialect: EG, Predicted Dialect: EG

Text: اليوفي حطات الع